In [1]:
import sys
print(sys.executable)
print(sys.prefix)

/opt/homebrew/Caskroom/miniforge/base/envs/torchLM/bin/python
/opt/homebrew/Caskroom/miniforge/base/envs/torchLM


In [2]:
import os
from datasets import load_dataset


from src.config import config_medgemma_4b_it_nih_cxr
from src.utils.fetch_data import load_data_chest_xray_pneumonia


/opt/homebrew/Caskroom/miniforge/base/envs/torchLM/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


## Load the NIH Chest X-Ray Pneumonia Data

In [3]:
dataset = load_data_chest_xray_pneumonia()

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 5216
    })
    validation: Dataset({
        features: ['image', 'label'],
        num_rows: 16
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 624
    })
})


## Fine tune medgemma-4b-it on the NIH Chest X-Ray Pneumonia Data


In [ ]:
from src.utils.data_utils import format_data_medgemma_nih_chest_x_ray
from src.utils.model_utils import load_model_and_processor
from src.config import config_medgemma_4b_it_nih_cxr
from src.fine_tune_medgemma_nih_pneumonia_chest_x_ray import fine_tune_medgemma_nih_pneumonia_chest_x_ray

# Format the dataset
formatted_dataset = dataset.map(format_data_medgemma_nih_chest_x_ray)

# Load base model and processor
model_id = "medgemma-4b-it"
base_model, processor = load_model_and_processor(
    model_id=config_medgemma_4b_it_nih_cxr.base_model_id,
    model_directory=config_medgemma_4b_it_nih_cxr.model_folder_base,
    model_kwargs=config_medgemma_4b_it_nih_cxr.model_kwargs,
)

# Fine-tune the model
fine_tune_medgemma_nih_pneumonia_chest_x_ray(
    model=base_model,
    processor=processor,
    data=formatted_dataset,
    base_model_directory=config_medgemma_4b_it_nih_cxr.model_folder_base,
    save_directory_adapters=config_medgemma_4b_it_nih_cxr.model_folder_pneumonia_ft_adapter,
    save_directory_full_model=config_medgemma_4b_it_nih_cxr.model_folder_pneumonia_ft_full,
    model_kwargs=config_medgemma_4b_it_nih_cxr.model_kwargs,
)

## Test the baseline model

In [22]:
from src.test_medgemma_49_it_nih_pneumonia_chest_x_ray import test_medgemma_4b_it_nih_pneumonia_chest_x_ray
from src.utils.prompt_utils import make_prompt_without_image


test_instance_base = test_medgemma_4b_it_nih_pneumonia_chest_x_ray(
    model_id=config_medgemma_4b_it_nih_cxr.base_model_id,
    model_folder=config_medgemma_4b_it_nih_cxr.model_folder_base,
    model_kwargs=config_medgemma_4b_it_nih_cxr.model_kwargs,
    max_new_tokens=250
)
messages = make_prompt_without_image(
        system_message=config_medgemma_4b_it_nih_cxr.prompt_template["system_message"],
        user_prompt=config_medgemma_4b_it_nih_cxr.prompt_template["user_prompt"])
print(f"System Message:{config_medgemma_4b_it_nih_cxr.prompt_template['system_message']}")
print(f"User Message:{config_medgemma_4b_it_nih_cxr.prompt_template['user_prompt']}")
print(f"True finding:{config_medgemma_4b_it_nih_cxr.condition_findings[dataset['test'][-7]['label']]}")

Loading checkpoint shards: 100%|██████████████████████████████████████████████████████████████████████| 2/2 [00:32<00:00, 16.48s/it]


Model and processor loaded from /Users/georgi/dev/meda/models/medgemma-4b-it
System Message:You are a medical AI expert analyzing chest X-rays     for pneumonia detection.
User Message:Analyze this chest X-ray image for signs of pneumonia     or normal lungs. Do you detect: 
 A: NORMAL
B: PNEUMONIA
True finding:B: PNEUMONIA


In [13]:
dataset["test"][-7]["image"].show()

In [14]:
base_assistant_message = test_instance_base.chat_v1(messages, dataset["test"][-7]["image"])

print(f"Baseline assistant message:{base_assistant_message}")

Generated Description: 
Based on the provided chest X-ray image, I would classify it as **A: NORMAL**.

Here's why:

*   **Lung Fields:** The lung fields appear relatively clear, with no obvious consolidation or infiltrates.
*   **Heart Size:** The heart size seems within normal limits.
*   **Mediastinum:** The mediastinum (the space between the lungs containing the heart, great vessels, etc.) appears normal.
*   **Bones:** The bony structures (ribs, clavicles, etc.) are intact.

While this is a preliminary assessment, and a definitive diagnosis requires a trained radiologist's interpretation, the image doesn't show any strong indicators of pneumonia.

Baseline assistant message:
Based on the provided chest X-ray image, I would classify it as **A: NORMAL**.

Here's why:

*   **Lung Fields:** The lung fields appear relatively clear, with no obvious consolidation or infiltrates.
*   **Heart Size:** The heart size seems within normal limits.
*   **Mediastinum:** The mediastinum (the space

## Test the fine-tuned model


In [17]:
test_instance_peft = test_medgemma_4b_it_nih_pneumonia_chest_x_ray(
    model_id="peft",
    model_folder=config_medgemma_4b_it_nih_cxr.model_folder_pneumonia_ft_full,
    model_kwargs=config_medgemma_4b_it_nih_cxr.model_kwargs,
    max_new_tokens=250
)
messages = make_prompt_without_image(
        system_message=config_medgemma_4b_it_nih_cxr.prompt_template["system_message"],
        user_prompt=config_medgemma_4b_it_nih_cxr.prompt_template["user_prompt"])
print(f"System Message:{config_medgemma_4b_it_nih_cxr.prompt_template['system_message']}")
print(f"User Message:{config_medgemma_4b_it_nih_cxr.prompt_template['user_prompt']}")
print(f"True finding:{config_medgemma_4b_it_nih_cxr.condition_findings[dataset['test'][-7]['label']]}")

Loading checkpoint shards: 100%|██████████████████████████████████████████████████████████████████████| 2/2 [00:32<00:00, 16.50s/it]
Some parameters are on the meta device because they were offloaded to the disk.


Model and processor loaded from /Users/georgi/dev/meda/models/medgemma-4b-it-pneumonia-finetuned-merged
System Message:You are a medical AI expert analyzing chest X-rays     for pneumonia detection.
User Message:Analyze this chest X-ray image for signs of pneumonia     or normal lungs. Do you detect: 
 A: NORMAL
B: PNEUMONIA
True finding:B: PNEUMONIA


In [20]:
ft_assistant_message = test_instance_peft.chat_v1(messages, dataset["test"][-7]["image"])
print(f"Fine-tuned assistant message:{ft_assistant_message}")

Generated Description: 
B: PNEUMONIA
Fine-tuned assistant message:
B: PNEUMONIA


## Test the augmented summary agent 

In [23]:
augmented_user_prompt = (config_medgemma_4b_it_nih_cxr.prompt_template["user_prompt"]
                             + f"\nIn your answer, please consider that a fine-tuned model ML model on a pneumonia"
                               f"\nchest X-ray dataset does predict the label **{ft_assistant_message}**  in this case. "
                               f"\nProvide a detailed explanation for your diagnosis.")
    
augmented_messages = make_prompt_without_image(
        system_message=config_medgemma_4b_it_nih_cxr.prompt_template["system_message"],
        user_prompt=augmented_user_prompt)
# augmented_assistant_message = test_instance_base.chat(augmented_messages)
augmented_assistant_message = test_instance_base.chat_v1(augmented_messages, dataset["test"][-7]["image"])

print(f"Augmented-tuned assistant message:{ft_assistant_message}")

Generated Description: 
Based on the chest X-ray image, I detect **B: PNEUMONIA**.

Here's a detailed explanation of my reasoning:

*   **Increased Opacity in the Right Lung:** The right lung appears more opaque (whiter) than the left lung. This increased opacity suggests consolidation, which is a hallmark of pneumonia. Consolidation refers to the filling of the air spaces in the lung with fluid, pus, or other substances.

*   **Possible Consolidation:** The increased opacity in the right lung could be due to consolidation, which is a common finding in pneumonia.

*   **Clinical Context:** While I cannot access the patient's clinical history, the presence of increased opacity in the right lung raises suspicion for pneumonia.

It's important to note that this is a preliminary assessment based on a single image. A definitive diagnosis requires a comprehensive evaluation, including the patient's clinical history, physical examination findings, and potentially additional imaging studies.

